<h1><center>Lab 2: Classification</center></h1>
<h3><center>A Deeper Analysis of Covid-19 Data</center></h3>
<p><center>DS 7331</center></p>
<p><center>Created by Sadik Aman, Dawn Bowerman, Zachary Harris, Alexandre Jasserme</center></p>

<p><center>Sections of this code was adapted from: 
    <li>https://github.com/jakemdrew/DataMiningNotebooks</li>

In [3]:
from __future__ import print_function

#Libraries
import plotly
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns
import pandas as pd
import sqlite3
from sklearn import datasets, linear_model
from sklearn.metrics import mean_squared_error, r2_score
from matplotlib.pyplot import figure
from sklearn.model_selection import TimeSeriesSplit
# run logistic regression and vary some parameters
from sklearn.linear_model import LogisticRegression
from sklearn.model_selection import KFold
from sklearn.linear_model import LinearRegression
from sklearn import metrics as mt
from sklearn.metrics import mean_absolute_error, make_scorer, mean_squared_error
from sklearn.model_selection import ShuffleSplit

df = pd.read_csv('Data/owid-covid-data_modified.csv') # read in the csv file

ParserError: Error tokenizing data. C error: Expected 1 fields in line 29, saw 376


## Data Preparation 

> [10 points] Define and prepare your class variables. Use proper variable representations
(int, float, one-hot, etc.). Use pre-processing methods (as needed) for dimensionality
reduction, scaling, etc. Remove variables that are not needed/useful for the analysis.

In [5]:
# Ideas from https://scikit-learn.org/stable/modules/cross_validation.html#timeseries-cv

# Sorting data frame by date column
df['date'] = pd.to_datetime(df['date']) # Converting data columnn to datetime

df = df.sort_values(by='date', ascending=True)

df.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 34965 entries, 0 to 34964
Data columns (total 26 columns):
 #   Column                          Non-Null Count  Dtype         
---  ------                          --------------  -----         
 0   continent                       34965 non-null  object        
 1   location                        34965 non-null  object        
 2   date                            34965 non-null  datetime64[ns]
 3   new_cases                       34965 non-null  float64       
 4   new_cases_smoothed              34965 non-null  float64       
 5   new_deaths                      34965 non-null  float64       
 6   new_deaths_smoothed             34965 non-null  float64       
 7   reproduction_rate               34965 non-null  float64       
 8   new_vaccinations_smoothed       34965 non-null  float64       
 9   new_people_vaccinated_smoothed  34965 non-null  float64       
 10  stringency_index                34965 non-null  float64       
 11  po

In [8]:
print ('Size of the dataset:', df.shape)
print ('Number of unique classes:', len(df.new_deaths_range))

Size of the dataset: (34965, 25)
Number of unique classes: 34965


In [9]:
# handle categorical variable
continent =pd.get_dummies(df,drop_first=True)
# dropping extra column
df= df.drop("continent",axis=1)
# concatenation of independent variables and new cateorical variable.
df=pd.concat([df,continent],axis=1)
df

,location,date,new_cases_smoothed,new_deaths,new_deaths_smoothed,reproduction_rate,new_vaccinations_smoothed,new_people_vaccinated_smoothed,stringency_index,population,...,stringency_range_40-50,stringency_range_50-60,stringency_range_60-70,stringency_range_70-80,stringency_range_80-90,stringency_range_90-100,new_cases_range_low,new_cases_range_medium,new_deaths_range_low,new_deaths_range_medium
0,Afghanistan,2021-07-01,1526.286,91.0,72.857,0.980000,14115.00000,13654.000000,46.300000,39835428,...,1,0,0,0,0,0,0,0,0,0
3515,Bermuda,2021-07-01,1.429,0.0,0.000,0.323548,430.00000,10.000000,37.960000,62092,...,0,0,0,0,0,0,1,0,1,0
13690,Guatemala,2021-07-01,1390.000,67.0,55.429,1.160000,8217.00000,7331.000000,51.850000,18249868,...,0,1,0,0,0,0,0,0,0,0
21275,Mexico,2021-07-01,4609.000,201.0,168.571,1.230000,522893.00000,348511.000000,44.440000,130262220,...,1,0,0,0,0,0,0,0,0,0
31265,Tajikistan,2021-07-01,21.714,0.0,0.000,1.530000,14170.00000,13967.000000,35.190000,9749625,...,0,0,0,0,0,0,1,0,1,0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
14429,Guyana,2022-01-01,99.000,0.0,1.286,1.960000,839.00000,199.000000,51.850000,790329,...,0,1,0,0,0,0,0,1,1,0
3144,Belgium,2022-01-01,9604.571,0.0,26.000,1.340000,45720.00000,958.000000,48.150000,11632334,...,1,0,0,0,0,0,1,0,1,0
31819,Thailand,2022-01-01,2715.429,10.0,21.429,1.180000,64643.66667,11719.500000,42.468421,69950844,...,1,0,0,0,0,0,0,0,0,0
26269,Qatar,2022-01-01,509.286,0.0,0.571,2.230000,7892.00000,8315.867257,33.486667,2930524,...,0,0,0,0,0,0,0,0,1,0


In [11]:
## KFold cross-validation


#separate the other attributes from the predicting attribute
covid_df = df.drop("new_deaths",axis=1)

#separate the predicting attribute into Y for model training 
covid_target = ["new_deaths"]


## Define variables for the for loop
kf = KFold(n_splits=10)
RMSE_sum=0
RMSE_length=10
X = np.array(covid_df)
y = np.array(covid_target)


In [13]:
  for loop_number, (train, test) in enumerate(kf.split(X)):
    
    
    ## Get Training Matrix and Vector

    training_X_array = X[train]
    training_y_array = y[train]

    ## Get Testing Matrix Values

    X_test_array = X[test]
    y_actual_values = y[test]

    ## Fit the Linear Regression Model

    lr_model = LinearRegression().fit(training_X_array, training_y_array)

    ## Compute the predictions for the test data

    prediction = lr_model.predict(X_test_array)      
    crime_probabilites = np.array(prediction)   

    ## Calculate the RMSE

    RMSE_cross_fold = RMSEcalc(crime_probabilites, y_actual_values)

    ## Add each RMSE_cross_fold value to the sum

    RMSE_sum=RMSE_cross_fold+RMSE_sum

## Calculate the average and print    

RMSE_cross_fold_avg=RMSE_sum/RMSE_length

print('The Mean RMSE across all folds is',RMSE_cross_fold_avg)

IndexError: index 3497 is out of bounds for axis 0 with size 1